### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 08 декабря 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 08 декабря, -4 балла после 08:30 15 декабря, -6 баллов после 08:30 22 декабря, -8 баллов после 08:30 29 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [243]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [244]:
class MyDecisionTreeClassifier(BaseEstimator):
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini', purity_level = 0.95, rel_criterion_limit=0):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        self.purity_level = purity_level
        self.rel_criterion_limit = rel_criterion_limit
        # Для последнего задания
        self.feature_importances = {}
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __div_ysamples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        p = np.array(np.unique(y, return_counts=True)[1])/y.shape[0]
#         print(p, y, "\n\n")
        if self.criterion == 'gini':
            def F(p):
                return 1 - (p*p).sum()
        elif self.criterion == 'entropic':
            def F(p):
                return (p*p.log()).sum()
        elif self.criterion == 'clf_loss':
            def F(p):
                return 1 - p.max()
        else:
            raise ValueError("Unsupported critetion")
        q_max = float('-inf')
#         x_uni = np.unique(x, axis=0)
        x_uni = x
        complex_x_uni = x_uni + np.tile(np.arange(x_uni.shape[1]), [x_uni.shape[0], 1])*1.0j
        def Q(val):
            y_left, y_right = self.__div_ysamples(x, y, int(val.imag), val.real)
            if y_left.shape[0] == 0 or y_right.shape[0] == 0:
                return float('-inf')
            p_left = np.array(np.unique(y_left, return_counts=True)[1])/y_left.shape[0]
            p_right = np.array(np.unique(y_right, return_counts=True)[1])/y_right.shape[0]
            q = - y_left.shape[0]/y.shape[0] * F(p_left) - y_right.shape[0]/y.shape[0] * F(p_right)
            return q
        Qs = np.vectorize(Q, otypes='f')(complex_x_uni)
        max_feature = Qs.argmax() % Qs.shape[1]
        max_threshold =  x_uni.ravel()[Qs.argmax()]
#         print(max_feature, max_threshold)
#         for ind, val in np.ndenumerate(np.unique(x, axis=1)):
#             y_left, y_right = self.__div_ysamples(x, y, ind[1], val)
#             if y_left.shape[0] == 0 or y_right.shape[0] == 0:
#                 continue
#             p_left = np.array(np.unique(y_left, return_counts=True)[1])/y_left.shape[0]
#             p_right = np.array(np.unique(y_right, return_counts=True)[1])/y_right.shape[0]
#             q = F(p)- y_left.shape[0]/y.shape[0] * F(p_left) - y_right.shape[0]/y.shape[0] * F(p_right)
#             if q > q_max:
#                 q_max = q
#                 max_threshold = val
#                 max_feature = ind[1]
        return max_feature, max_threshold, F(p)-Qs.max()
        
    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
#         print(x.shape, y.shape)
        counts = np.unique(y, return_counts=True)
        max_ratio = counts[1].max()/y.shape[0]
        predict = counts[0][counts[1].argmax()]
        if (x.shape[0] < self.min_samples_split) or depth >= self.max_depth or \
                max_ratio > self.purity_level:    
            self.tree[node_id] = (self.__class__.LEAF_TYPE, predict, max_ratio)
#             print(y,'\n', counts, '\n', self.tree[node_id], '\n\n')
        else:
            feature_id, threshold, q = self.__find_threshold(x, y)
            if q < self.rel_criterion_limit:
                counts = np.unique(y, return_counts=True)
                self.tree[node_id] = (self.__class__.LEAF_TYPE, counts[0][counts[1].argmax()], counts[1].max()/y.shape[0])
                return
            self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, feature_id, threshold)
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, feature_id, threshold)
            if feature_id in self.feature_importances.keys():
                self.feature_importances[feature_id] += q
            else:
                self.feature_importances[feature_id] = q
            self.__fit_node(x_left, y_left, 2*node_id+1, depth+1)
            self.__fit_node(x_right, y_right, 2*node_id+2, depth+1)
        # Ваш код здесь
        pass
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance(self):
        """
        Возвращает важность признаков
        """
        return self.feature_importances
        # Ваш код здесь
        pass

In [163]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [164]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [165]:
# print(np.unique(y_train, return_counts=True)[1].argmax())
# counts = np.unique(y_train, return_counts=True)
# counts[0][counts[1].argmax()]
criterion = 'gini'
print(X_train.shape)
p = np.array(np.unique(y_train, return_counts=True)[1])/y_train.shape[0]
if criterion == 'gini':
    def F(p):
        return 1 - (p*p).sum()
elif criterion == 'entropic':
    def F(p):
        return 1 - (p*p.log()).sum()
elif criterion == 'clf_loss':
    def F(p):
        return 1 - p.max()
else:
    raise ValueError("Unsupported critetion")

F(p)

(160, 13)


0.6580468749999999

In [166]:
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [167]:
print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))
print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

1.0
1.0


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [168]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.97 ms


DecisionTreeClassifier()

In [169]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.42 s, sys: 46.9 ms, total: 1.47 s
Wall time: 2.43 s


In [170]:
len(my_clf.tree)

19

## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [171]:
df = pd.read_csv('./Speed Dating Data.csv', encoding='latin1')

In [172]:
df.info(max_cols=200)
df
# df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 195 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   iid       8378 non-null   int64  
 1   id        8377 non-null   float64
 2   gender    8378 non-null   int64  
 3   idg       8378 non-null   int64  
 4   condtn    8378 non-null   int64  
 5   wave      8378 non-null   int64  
 6   round     8378 non-null   int64  
 7   position  8378 non-null   int64  
 8   positin1  6532 non-null   float64
 9   order     8378 non-null   int64  
 10  partner   8378 non-null   int64  
 11  pid       8368 non-null   float64
 12  match     8378 non-null   int64  
 13  int_corr  8220 non-null   float64
 14  samerace  8378 non-null   int64  
 15  age_o     8274 non-null   float64
 16  race_o    8305 non-null   float64
 17  pf_o_att  8289 non-null   float64
 18  pf_o_sin  8289 non-null   float64
 19  pf_o_int  8289 non-null   float64
 20  pf_o_fun  8280 non-null   flo

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,10.0,5,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8374,552,22.0,1,44,2,21,22,13,10.0,4,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8375,552,22.0,1,44,2,21,22,19,10.0,10,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0
8376,552,22.0,1,44,2,21,22,3,10.0,16,...,8.0,5.0,7.0,6.0,7.0,9.0,5.0,9.0,5.0,6.0


In [173]:
df.iid.nunique()
df = df.drop(['id'], axis=1)
df = df.drop(['idg'], axis=1)
df = df.drop(['condtn'], axis=1)
df = df.drop(['zipcode', 'from', 'career', 'tuition', 'undergra', 'mn_sat'], axis=1)
df = df.drop(['expnum'], axis=1)

In [174]:
feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

In [175]:
idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
temp.loc[idx, ]

,iid,wave,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,totalsum
312,28,2,NaN,NaN,NaN,NaN,NaN,NaN,0.0
828,58,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
838,59,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
918,67,3,20.0,15.0,20.0,20.0,5.0,10.0,90.0
1530,105,4,30.0,15.0,20.0,20.0,0.0,5.0,90.0
5004,339,13,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5014,340,13,NaN,NaN,NaN,NaN,NaN,NaN,0.0
5114,346,14,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7221,489,19,20.0,10.0,20.0,20.0,20.0,0.0,90.0
7586,517,21,15.0,20.0,20.0,20.0,5.0,10.0,90.0


In [176]:
idx = ((temp.wave >= 6) & (temp.wave <= 9))
temp.loc[idx, ]

,iid,wave,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,totalsum
1846,132,6,16.67,16.67,16.67,16.67,16.67,16.67,100.02
1851,133,6,12.77,19.15,17.02,17.02,14.89,19.15,100.00
1856,134,6,6.67,20.00,20.00,17.78,20.00,15.56,100.01
1861,135,6,18.18,22.73,18.18,13.64,13.64,13.64,100.01
1866,136,6,NaN,NaN,NaN,NaN,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...
3308,229,9,21.43,16.67,21.43,16.67,11.90,11.90,100.00
3328,230,9,23.81,23.81,23.81,23.81,2.38,2.38,100.00
3348,231,9,18.60,20.93,23.26,23.26,2.33,11.63,100.01
3368,232,9,17.78,17.78,17.78,17.78,13.33,15.56,100.01


In [177]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

In [178]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1']].sum(axis=1)
df.loc[:, ['attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1']] = \
(df.loc[:, ['attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2']].sum(axis=1)
df.loc[:, ['attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2']] = \
(df.loc[:, ['attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2']].T/df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100


df = df.drop(['temp_totalsum'], axis=1)
# df = df.drop()


In [179]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)
                                 
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)
                                   
        
df_female.columns = df_female.columns + '_f'

In [180]:
df_final = df_male.merge(df_female,left_on=['pid'], right_on=['iid_f'])
df_final = df_final.fillna(df_final.mean())
df_final = df_final.drop(['income', 'income_f', 'field', 'field_f'], axis=1)
df_final.info(max_cols=400)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4184 entries, 0 to 4183
Data columns (total 361 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   iid         4184 non-null   int64  
 1   wave        4184 non-null   int64  
 2   round       4184 non-null   int64  
 3   position    4184 non-null   int64  
 4   positin1    4184 non-null   float64
 5   order       4184 non-null   int64  
 6   partner     4184 non-null   int64  
 7   pid         4184 non-null   float64
 8   match       4184 non-null   int64  
 9   int_corr    4184 non-null   float64
 10  samerace    4184 non-null   int64  
 11  age_o       4184 non-null   float64
 12  race_o      4184 non-null   float64
 13  pf_o_att    4184 non-null   float64
 14  pf_o_sin    4184 non-null   float64
 15  pf_o_int    4184 non-null   float64
 16  pf_o_fun    4184 non-null   float64
 17  pf_o_amb    4184 non-null   float64
 18  pf_o_sha    4184 non-null   float64
 19  dec_o       4184 non-null 

In [181]:
y = np.array(df_final.match)
X = np.array(df_final.drop(['match'], axis=1), dtype='float64')

In [160]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
my_clf.fit(X_train, y_train)
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

KeyboardInterrupt: 

In [ ]:
my_clf.tree

In [189]:
# df_final = df_final.drop(['dec_o', 'dec', 'dec_o_f', 'dec_f'], axis=1)
y = np.array(df_final.match)
X = np.array(df_final.drop(['match'], axis=1), dtype='float64')
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
my_clf.fit(X_train, y_train)
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

0.8615751789976134

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, GroupKFold
my_clf = MyDecisionTreeClassifier(min_samples_split=2, purity_level = 1)
group_kfold = GroupKFold(n_splits=5)
cross_val_score(my_clf, X, y, cv=5, scoring='accuracy')

In [ ]:
max_score = 0
depths = [3, 4, 5, 6, 7, 8, 9, 10]
sample_split = [2, 5, 10, 20]
purity_levels = np.arange(0.85, 1.01, 0.01)

for depth in depths:
    for min_sample in samples_split:
        for purity_level in purity_levels:
            my_clf = MyDecisionTreeClassifier(min_samples_split=min_sample, purity_level = purity_level, depth=depth)
            cur_score = cross_val_score(
                my_clf, X, y, scoring=make_scorer('accuracy'))
            print(cur_score.mean())
            if cur_score.mean() > max_score:
                max_score = cur_score.mean()
                params = (depth, min_sample, purity_level)

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [206]:
feat_dict = my_clf.get_feature_importance()
indexes = sorted(feat_dict, key=feat_dict.get, reverse=True)
scores = sorted(feat_dict.values(), reverse=True)
# print(indexes)
features = (np.array(list(df_final.columns))[indexes])
for i in range(len(indexes)):
    print(scores[i], features[i])


1.5298741453903446 met
1.42852625471541 shar_o
1.2152064371330114 shar
0.9607985927378481 met_f
0.9533172491273615 you_call_f
0.9012302036322297 amb5_1
0.754687488079071 tv
0.7362078105360954 dining
0.717320592863762 you_call
0.7104942952166635 theater
0.6698141784118119 sinc1_2_f
0.66853016357363 iid
0.5245794822644474 pf_o_sha
0.4691052901136422 positin1
0.31999999999999984 field_cd
0.2790011951969269 race_o
0.2553336858027533 like
0.19608159590951035 intel2_1


In [223]:
np.append(features, ('match'))
small_df_final = df_final[list(features)+['match']]
y = np.array(small_df_final.match)
X = np.array(small_df_final.drop(['match'], axis=1), dtype='float64')
my_clf = MyDecisionTreeClassifier(min_samples_split=2, purity_level = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
my_clf.fit(X_train, y_train)
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

0.8210023866348448

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

